In [86]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
from Ge_calculation import GAP, MD_run
import matplotlib.pyplot as plt
from ase.io.cfg import read_cfg
from ase.io.extxyz import write_extxyz
from ase.neighborlist import neighbor_list
import numpy as np
import re
from sys import argv

import os
from ase import build, Atoms
from quippy.potential import Potential
from Ge_analysis import *
from ase.io.proteindatabank import read_proteindatabank, write_proteindatabank
from ase.io.lammpsdata import write_lammps_data
from ase.neighborlist import neighbor_list
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit
from scipy.stats import gaussian_kde
from glob import glob
import sys
from ase import build
from ase.io import Trajectory
from ase.io import read
from ase.eos import EquationOfState
from ase.visualize import view
from copy import deepcopy
from ase.io.castep import read_castep_castep, read_param
from ase.calculators.castep import Castep
from ase.io import read
from ase.io.extxyz import read_xyz
import pandas as pd

from ase.io.castep import write_castep_cell

In [91]:
from ase_scripts.make_castep import make_castep
import pymatgen.ext.matproj as mp
import pymatgen.core.periodic_table as pt
import pymatgen.io.ase as pase

In [25]:
with mp.MPRester() as m:
    results_master = m.query(criteria={"pretty_formula":'Si'},
                             properties=["exp.tags", "icsd_ids", "task_id", "density", "final_structure"])

[{'exp.tags': ['Silicon'],
  'icsd_ids': [189396],
  'task_id': 'mp-1196961',
  'density': 1.9845847893263202,
  'final_structure': Structure Summary
  Lattice
      abc : 10.221459979417617 10.221459979417617 10.388793
   angles : 90.0 90.0 119.99999986677838
   volume : 939.9864436293609
        A : 5.11073 -8.852044 0.0
        B : 5.11073 8.852044 0.0
        C : 0.0 0.0 10.388793
  PeriodicSite: Si (7.0110, -1.8535, 7.1433) [0.7906, 0.5812, 0.6876]
  PeriodicSite: Si (3.2104, 1.8535, 7.1433) [0.2094, 0.4188, 0.6876]
  PeriodicSite: Si (3.2104, -1.8535, 3.2455) [0.4188, 0.2094, 0.3124]
  PeriodicSite: Si (7.0110, 1.8535, 3.2455) [0.5812, 0.7906, 0.3124]
  PeriodicSite: Si (3.2104, 1.8535, 3.2455) [0.2094, 0.4188, 0.3124]
  PeriodicSite: Si (7.0110, -1.8535, 3.2455) [0.7906, 0.5812, 0.3124]
  PeriodicSite: Si (7.0110, 1.8535, 7.1433) [0.5812, 0.7906, 0.6876]
  PeriodicSite: Si (3.2104, -1.8535, 7.1433) [0.4188, 0.2094, 0.6876]
  PeriodicSite: Si (3.1679, -5.4869, 8.4429) [0.6198, 0.

In [30]:
len(results_master)

39

In [34]:
def ang_cm(mass, volume):
    return mass * 1.661 * 10**(-24) / (volume * 10**(-24))

results = [i for i in results_master
           if i['density'] > ang_cm(pt.Element('Si').atomic_mass, 16)]
len(results)

10

In [38]:
print([[i['exp.tags'], results.index(i)] for i in results])

[[['Silicon - VI', 'High pressure experimental phase'], 0], [['High pressure experimental phase', 'Silicon - HP'], 1], [[], 2], [['High pressure experimental phase', 'Silicon - HP'], 3], [['Silicon - VI', 'High pressure experimental phase'], 4], [['High pressure experimental phase', 'Silicon - HP', 'Silicon - II, HP'], 5], [[], 6], [['Silicon - HP', 'Silicon - V, HP', 'High pressure experimental phase'], 7], [['Manganese silicide (0.57/1)'], 8], [[], 9]]


In [88]:
ats = [pase.AseAtomsAdaptor.get_atoms(i['final_structure']) for i in results]
for i in range(len(ats)):
    ats[i].info['mp_id'] = results[i]['task_id']
    ats[i].info['icsd_id'] = results[i]['icsd_ids']
    ats[i].info['exp.tags'] = results[i]['exp.tags']

In [92]:
make_castep(ats, '/Users/Moji/Documents/Summer20/Ge/Structure_databases/crystals/Si_hp_crystals_mp',
            param='template.param', kpt_spacing=0.03)

0 is done


In [ ]:
s = m.query(criteria={"pretty_formula":'Si'}

In [98]:
[i.info['exp.tags'] for i in ats]

[['Silicon - VI', 'High pressure experimental phase'],
 ['High pressure experimental phase', 'Silicon - HP'],
 [],
 ['High pressure experimental phase', 'Silicon - HP'],
 ['Silicon - VI', 'High pressure experimental phase'],
 ['High pressure experimental phase', 'Silicon - HP', 'Silicon - II, HP'],
 [],
 ['Silicon - HP', 'Silicon - V, HP', 'High pressure experimental phase'],
 ['Manganese silicide (0.57/1)'],
 []]

In [99]:
view(ats[7])